<a href="https://colab.research.google.com/github/rafaelpivetta/tech-challenge-fase3/blob/main/Running_LLama3_Chapter3_a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
!pip install llama-cpp-python
!pip install huggingface_hub
!pip install datasets
!pip install trl
!pip install evaluate
!pip install peft
!pip install rouge_score

In [10]:
from llama_cpp import Llama
from datasets import load_dataset, Dataset

from datasets import load_dataset

ds = load_dataset("bitext/Bitext-customer-support-llm-chatbot-training-dataset",
                  split="train")
print(ds.column_names)
print(ds.shape)

['flags', 'instruction', 'category', 'intent', 'response']
(26872, 5)


In [11]:
first_thousand_points = ds[:1000]
ds = Dataset.from_dict(first_thousand_points)

In [12]:
import pprint
pprint.pprint(ds[0])

{'category': 'ORDER',
 'flags': 'B',
 'instruction': 'question about cancelling order {{Order Number}}',
 'intent': 'cancel_order',
 'response': "I've understood you have a question regarding canceling order "
             "{{Order Number}}, and I'm here to provide you with the "
             'information you need. Please go ahead and ask your question, and '
             "I'll do my best to assist you."}


In [13]:
def merge_example(row):
  row['conversation'] = f"Query: {row['instruction']}\nResponse: {row['response']}"
  return row

# Call the ds map method to apply our preprocessing function to all rows
ds = ds.map(merge_example) # Aplica a função para cada linha
print(ds[0]['conversation'])

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Query: question about cancelling order {{Order Number}}
Response: I've understood you have a question regarding canceling order {{Order Number}}, and I'm here to provide you with the information you need. Please go ahead and ask your question, and I'll do my best to assist you.


In [20]:
from transformers import TrainingArguments
from trl import SFTTrainer
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Maykeye/TinyLLama-v0"

model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # right pad sequences

training_arguments = TrainingArguments(output_dir='./results',
    per_device_train_batch_size=1,
    learning_rate=2e-3,
    max_grad_norm=0.5,
    max_steps=200,
    gradient_accumulation_steps=2,
    save_steps=2)

SFTTrainer(model=model,
           tokenizer=tokenizer,
           args=training_arguments,
           train_dataset=ds,
           dataset_text_field='conversation',
           max_seq_length=250,).train()




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:407: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss


TrainOutput(global_step=200, training_loss=2.0258209228515627, metrics={'train_runtime': 165.7285, 'train_samples_per_second': 2.414, 'train_steps_per_second': 1.207, 'total_flos': 1441641262464.0, 'train_loss': 2.0258209228515627, 'epoch': 0.4})

Avaliação do resultado

In [23]:
import evaluate

rouge = evaluate.load('rouge')
predictions = ["hello there", "general kenobi"]
references = ["hello there", "master yoda"]
results = rouge.compute(predictions=predictions, references=references)
print(results)

def generate_predictions_and_references(ds):
  predictions = []
  references = []
  for row in ds:
    inputs = tokenizer.encode(row["instruction"], return_tensors="pt")
    outputs = model.generate(inputs)
    decoded_output = tokenizer.decode(outputs[0, inputs.shape[1]:], skip_special_tokens=True)
    references += [row["response"]]
    predictions += [decoded_output]
  return references, predictions

references, predictions = generate_predictions_and_references(ds)
results = rouge.compute(predictions=predictions, references=references)
print(results)

{'rouge1': 0.5, 'rouge2': 0.5, 'rougeL': 0.5, 'rougeLsum': 0.5}


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'rouge1': 0.030830560278825348, 'rouge2': 0.010015005271346498, 'rougeL': 0.03025365178645173, 'rougeLsum': 0.030456947099464206}


Fine-tune com Lora

In [26]:
from peft import LoraConfig
lora_config = LoraConfig(
    r=12,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['q_proj', 'v_proj']
)

SFTTrainer(model=model,
           tokenizer=tokenizer,
           args=training_arguments,
           train_dataset=ds,
           dataset_text_field='conversation',
           max_seq_length=250,
           peft_config=lora_config).train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:407: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss


TrainOutput(global_step=200, training_loss=0.9910741424560547, metrics={'train_runtime': 127.1835, 'train_samples_per_second': 3.145, 'train_steps_per_second': 1.573, 'total_flos': 1455409081728.0, 'train_loss': 0.9910741424560547, 'epoch': 0.4})

In [2]:
# Load model directly
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("nvidia/Llama3-ChatQA-1.5-8B")
# model = AutoModelForCausalLM.from_pretrained("nvidia/Llama3-ChatQA-1.5-8B")